In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, f1_score, precision_score
from keras.models import load_model
import numpy as np

In [ ]:
def preprocess_val_data(X, mean_X, std_X):
    
    X = X.astype(float)
    X -= mean_X
    X /= std_X
    return X

def evaluate_model(model_path, valX, valY):
    """Evaluation function for the model.
    
    # Arguments
        model_path: path to the trained model that is to be evaluated. Input size for the model is 240x320x3
        valX: 4D numpy array containing images in the format NxHxWx3. Where 
            N is number of images, H is height of image and it should be 240 pixels, 
            W is width of image and it should be 320 pixels. 
        valY: 1D numpy array containing labels (0:'SCC_below_56', 1:'SCC_56-132', 2:'SCC_above_132') 
        for the images. array([1,2,0,0,....])
    # Returns
        confusion matrix, recall for each class, precision for each class and f1 score for each class
    
    """
    
    mean_blue = np.array([0.35233836, 2.25305321, 9.56535592])
    std_blue = np.array([ 3.85356075, 10.1685023 , 28.70528262])
    mean_red = np.array([32.5874143 ,  3.37447883,  1.28614109])
    std_red = np.array([51.23512265,  9.75799306,  4.83126132])
    
    valX_blue = valX[:,:,:320,:]
    valX_red = valX[:,:,320:,:]
    
    valX_blue = preprocess_val_data(valX_blue, mean_blue, std_blue)
    valX_red = preprocess_val_data(valX_red, mean_red, std_red)
    
    model = load_model(model_path)
    y_prob = model.predict([valX_blue, valX_red])
    y_pred = y_prob.argmax(axis=-1)
    
    conf_matrix = confusion_matrix(valY, y_pred)
    recall = recall_score(valY, y_pred, average=None)
    precision = precision_score(valY, y_pred, average=None)
    f1 = f1_score(valY, y_pred, average=None)
    
    return conf_matrix, recall, precision, f1